## Download data from Zenodo

In [1]:
import requests
import os
datasets_to_download_zenodo = [('refwug', 'German', '1.1.0', 'https://zenodo.org/records/5791269/files/refwug.zip?download=1', ''), 
                            ('durel', 'German', '3.0.0', 'https://zenodo.org/records/5784453/files/durel.zip?download=1', ''),
                            ('surel', 'German', '3.0.0', 'https://zenodo.org/records/5784569/files/surel.zip?download=1', ''), 
                            ('chiwug', 'Chinese', '1.0.0', 'https://zenodo.org/records/10023263/files/chiwug.zip?download=1', ''),
                            ('dwug_de', 'German', '3.0.0', 'https://zenodo.org/records/14028509/files/dwug_de.zip?download=1', ''),
                            ('dwug_en', 'English', '3.0.0', 'https://zenodo.org/records/14028531/files/dwug_en.zip?download=1', ''),
                            ('dwug_sv', 'Swedish', '3.0.0', 'https://zenodo.org/records/14028906/files/dwug_sv.zip?download=1', ''),
                            ('dwug_de_resampled', 'German', '1.0.0', 'https://zenodo.org/records/12670698/files/dwug_de_resampled.zip?download=1', ''),
                            ('dwug_en_resampled', 'English', '1.0.0', 'https://zenodo.org/records/14025941/files/dwug_en_resampled.zip?download=1', ''),
                            ('dwug_sv_resampled', 'Swedish', '1.0.0', 'https://zenodo.org/records/14026615/files/dwug_sv_resampled.zip?download=1', ''),
                            ('discowug', 'German', '2.0.0', 'https://zenodo.org/records/14028592/files/discowug.zip?download=1', ''),
                            ('dwug_es', 'Spanish', '4.0.2', 'https://zenodo.org/records/14891659/files/dwug_es.zip?download=1', ''),
                            ('diawug', 'Spanish', '1.1.2', 'https://zenodo.org/records/14891461/files/diawug.zip?download=1', ''),
                                ]

if not os.path.exists('data/'):
    os.makedirs('data/')      

for name, language, version, link, path_to_data in datasets_to_download_zenodo:
    r = requests.get(link, allow_redirects=True)
    f = 'data/' + name + '.zip'
    open(f, 'wb').write(r.content)

## Download data from github

In [ ]:
'''
datasets_to_download_github = [('rusemshift_1', 'Russian', '', 'https://github.com/Garrafao/RuSemShift/archive/refs/heads/correct-indices.zip', 'RuSemShift-correct-indices/rusemshift_1/DWUG/'), 
                                ('rusemshift_2', 'Russian', '', 'https://github.com/Garrafao/RuSemShift/archive/refs/heads/correct-indices.zip', 'RuSemShift-correct-indices/rusemshift_2/DWUG/'), 
                                ('rushifteval1', 'Russian', '', 'https://github.com/Garrafao/rushifteval_public/archive/refs/heads/correct-indices.zip', 'rushifteval_public-correct-indices/durel/rushifteval1/'), 
                                ('rushifteval2', 'Russian', '', 'https://github.com/Garrafao/rushifteval_public/archive/refs/heads/correct-indices.zip', 'rushifteval_public-correct-indices/durel/rushifteval2/'), 
                                ('rushifteval3', 'Russian', '', 'https://github.com/Garrafao/rushifteval_public/archive/refs/heads/correct-indices.zip', 'rushifteval_public-correct-indices/durel/rushifteval3/'), 
                                ('rudsi', 'Russian', '', 'https://github.com/Garrafao/RuDSI/archive/refs/heads/correct-indices.zip', 'RuDSI-correct-indices/'), 
                                ('nordiachange1', 'Norwegian', '', 'https://github.com/Garrafao/nor_dia_change/archive/refs/heads/correct-indices.zip', 'nor_dia_change-correct-indices/subset1/'), 
                                ('nordiachange2', 'Norwegian', '', 'https://github.com/Garrafao/nor_dia_change/archive/refs/heads/correct-indices.zip', 'nor_dia_change-correct-indices/subset2/')
                              ]
for name, language, version, link, path_to_data in datasets_to_download_github:
    r = requests.get(link, allow_redirects=True)
    f = 'data/' + name + '.zip'
    open(f, 'wb').write(r.content)
'''    

In [5]:
datasets_all = datasets_to_download_zenodo

'''
datasets_all = datasets_to_download_zenodo + datasets_to_download_github

languages_global = ['German', 'English', 'Swedish', 'Spanish', 'Chinese', 'Russian', 'Norwegian']
'''

"\ndatasets_all = datasets_to_download_zenodo + datasets_to_download_github\n\nlanguages_global = ['German', 'English', 'Swedish', 'Spanish', 'Chinese', 'Russian', 'Norwegian']\n"

## Unzip data and remove superfluous files

In [6]:
import zipfile
import shutil

for name, language, version, link, path_to_data in datasets_all:
    if not os.path.exists('data/' + name):
        os.makedirs('data/' + name)
    else:
        shutil.rmtree('data/' + name)        
        os.makedirs('data/' + name)
    if path_to_data == '':
        with zipfile.ZipFile('data/' + name + '.zip') as z:
            z.extractall('data/temp')
        dest = shutil.move('data/temp/' + name + '/data', 'data/' + name)  
        if os.path.exists('data/temp/' + name + '/clusters'):
            dest = shutil.move('data/temp/' + name + '/clusters', 'data/' + name)  
    else:
        with zipfile.ZipFile('data/' + name + '.zip') as z:
            z.extractall('data/temp/' + name)
        dest = shutil.move('data/temp/' + name + '/' + path_to_data + '/data/', 'data/' + name + '/data')  
        if os.path.exists('data/temp/' + name + '/' + path_to_data + '/clusters/'):
            dest = shutil.move('data/temp/' + name + '/' + path_to_data + '/clusters/', 'data/' + name + '/clusters')  
    shutil.rmtree('data/temp/' + name)        

In [7]:
import pandas as pd
from pathlib import Path
import unicodedata
import numpy as np

'''
# Load datasets into data frame
df_judgments = pd.DataFrame()
j = 0
i2lemma2name_judgments = []
for name, language, version, link, path_to_data in datasets_all:
    print(name)
    i = 0
    for p in Path('data/'+name+'/data').glob('*/judgments.csv'):
        #print(p)
        lemma = str(p).split('/')[-2]        
        lemma = unicodedata.normalize('NFC', lemma)
        df = pd.read_csv(p, delimiter='\t', quoting=3, na_filter=False)
        df['dataset'] = name
        df['language'] = language
        df['annotator'] = df['annotator'].astype(str) + '-' + name # make sure annotators are unique across datasets
        if name in ['chiwug']:            
            df['identifier1'] = df['identifier1'].astype(str) + '-' + str(i) # make sure identifiers are unique across words
            df['identifier2'] = df['identifier2'].astype(str) + '-' + str(i) # make sure identifiers are unique across words
        if name in ['rusemshift_1', 'rusemshift_2']: # only done four judgments for those datasets which will not be mapped later
            # don't do this for the German data where same identifiers mean same use
            df['identifier1'] = df['identifier1'].astype(str) + '-' + str(j) # make sure identifiers are unique across datasets
            df['identifier2'] = df['identifier2'].astype(str) + '-' + str(j) # make sure identifiers are unique across datasets
        df['judgment'] = df['judgment'].astype(float)
        #if df['judgment'].isnull().values.any():            
        #    display(df[df['judgment'].isnull()])
        df_judgments = pd.concat([df_judgments, df])
        i2lemma2name_judgments.append((i,lemma,name))
        i+=1
    j+=1
'''    
df_uses = pd.DataFrame()
j = 0
i2lemma2name_uses = []
for name, language, version, link, path_to_data in datasets_all:
    if not os.path.exists('data/' + name +'/data_durel'):
        os.makedirs('data/' + name +'/data_durel')
    i = 0
    for p in Path('data/'+name+'/data').glob('*/uses.csv'):
        #print(p)
        lemma = str(p).split('/')[-2]        
        lemma = unicodedata.normalize('NFC', lemma)
        shutil.copyfile(p, 'data/' + name +'/data_durel/' + lemma + '_uses.csv')                
        continue
        df = pd.read_csv(p, delimiter='\t', quoting=3, na_filter=False)
        df['dataset'] = name
        df['language'] = language
        if name in ['chiwug']:
            df['identifier'] = df['identifier'].astype(str) + '-' + str(i) # make sure identifiers are unique across words
            df['lemma'] = df['lemma'].apply(lambda x: unicodedata.normalize('NFC', x))
        if name in ['rushifteval1', 'rushifteval2', 'rushifteval3', 'rusemshift_1', 'rusemshift_2']:
            df['identifier'] = df['identifier'].astype(str) + '-' + str(j) # make sure identifiers are unique across datasets
        df_uses = pd.concat([df_uses, df])        
        i2lemma2name_uses.append((i,lemma,name))
        i+=1
    j+=1
'''    
df_clusters = pd.DataFrame()
j = 0
i2lemma2name_clusters = []
for name, language, version, link, path_to_data in datasets_all:
    i = 0    
    for p in Path('data/'+name+'/data').glob('*/uses.csv'): # read in uses to have same order of data for clusters
        lemma = str(p).split('/')[-2]        
        lemma = unicodedata.normalize('NFC', lemma)
        if os.path.exists('/'.join(str(p).split('/')[:-3] + ['clusters'])):
            p = '/'.join(str(p).split('/')[:-3] + ['clusters/opt/'+lemma+'.csv'])
            #print(p)
            df = pd.read_csv(p, delimiter='\t', quoting=3, na_filter=False)
            df['dataset'] = name
            df['language'] = language
            df['lemma'] = lemma
            if name in ['chiwug']:
                df['identifier'] = df['identifier'].astype(str) + '-' + str(i) # make sure identifiers are unique across words
            if name in ['rushifteval1', 'rushifteval2', 'rushifteval3', 'rusemshift_1', 'rusemshift_2']:
                df['identifier'] = df['identifier'].astype(str) + '-' + str(j) # make sure identifiers are unique across datasets
            df_clusters = pd.concat([df_clusters, df])        
        i2lemma2name_clusters.append((i,lemma,name))
        i+=1
    j+=1        
'''

"    \ndf_clusters = pd.DataFrame()\nj = 0\ni2lemma2name_clusters = []\nfor name, language, version, link, path_to_data in datasets_all:\n    i = 0    \n    for p in Path('data/'+name+'/data').glob('*/uses.csv'): # read in uses to have same order of data for clusters\n        lemma = str(p).split('/')[-2]        \n        lemma = unicodedata.normalize('NFC', lemma)\n        if os.path.exists('/'.join(str(p).split('/')[:-3] + ['clusters'])):\n            p = '/'.join(str(p).split('/')[:-3] + ['clusters/opt/'+lemma+'.csv'])\n            #print(p)\n            df = pd.read_csv(p, delimiter='\t', quoting=3, na_filter=False)\n            df['dataset'] = name\n            df['language'] = language\n            df['lemma'] = lemma\n            if name in ['chiwug']:\n                df['identifier'] = df['identifier'].astype(str) + '-' + str(i) # make sure identifiers are unique across words\n            if name in ['rushifteval1', 'rushifteval2', 'rushifteval3', 'rusemshift_1', 'rusemshift_2